In [ ]:
import os
# import cv2
import pandas as pd
from time import sleep
# from ultralytics import YOLO
import xml.etree.ElementTree as ET

NORMAL = False

INCL_ANNO = True
INCL_YOLO2 = True
INCL_YOLO4 = True
INCL_R3p1 = False

# no point in including annotations on normal video
INCL_ANNO = INCL_ANNO and not NORMAL

MIN_CONF = 0.25

# Open video list
if NORMAL:
    video_list = os.listdir('/notebooks/data/Datasets/CPTAD/Videos_Normal/Test/')
else:
    video_list = pd.read_csv('/notebooks/Thesis/data_yolo/test_videos.txt', sep=" ", header=None)

# Open annotations
if INCL_ANNO:
    tree = ET.parse('/notebooks/Thesis/1_Preprocessing/annotations_reformatted.xml')
    videos = tree.getroot()

def overlap(boxA, boxB):
    areaA = float((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    areaB = float((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    intAB = [max(boxA[0], boxB[0]), # xtl
             max(boxA[1], boxB[1]), # ytl
             min(boxA[2], boxB[2]), # xbr
             min(boxA[3], boxB[3])] # ybr

    if intAB[0] > intAB[2] or intAB[1] > intAB[3]:
        return 0

    areaAB = float((intAB[2] - intAB[0]) * (intAB[3] - intAB[1]))

    return areaAB / min(areaA, areaB)

for i in range(len(video_list)):

    # get video path
    if NORMAL:
        video_name = video_list[i]
        video_path = f"/notebooks/data/Datasets/CPTAD/Videos_Normal/Test/{video_name}"
    else:
        video_id = video_list.iloc[i, 0]
        video_name = video_list.iloc[i, 1]
        video_path = f"/notebooks/data/Datasets/CPTAD/Videos/{video_name}"

    # get tracks from annotations
    if INCL_ANNO:
        video = videos.find(f".//video[@taskid='{video_id}']")
        tracks = video.find('tracks')

    if INCL_YOLO4:
        yolo_boxes4 = pd.read_csv(f"/notebooks/Thesis/results/yolo_train4/{video_name[:-4]}_{video_id}.csv")

    if INCL_YOLO2:
        yolo_boxes2 = pd.read_csv(f"/notebooks/Thesis/results/yolo_train2/{video_name[:-4]}_{video_id}.csv")

    print(video_path)

    yolo4_detList = [] # list of [[num_correct, num_incorrect], [num_correct, num_incorrect], ...] indexed by frame     
    # Loop through the video frames
    for framenum in range(int(video.find('length'))):
       
        # Read a frame from the video
        framenum = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        num_correct = 0
        num_incorrect = 0
 
        # show annotations
        crash_frames = tracks.findall(f".//track[@Element1='Car'][@Element2='Car']/frame[@frame='{framenum}']")

        yolo_filtered = yolo_boxes4.loc[(yolo_boxes4['frame'] == framenum) & (yolo_boxes4['conf'] > MIN_CONF)]
        for j in range(len(yolo_filtered)):
            row = yolo_filtered.iloc[j]
            boxB = [(row['x'] - row['w']/2), # xtl
                    (row['y'] - row['h']/2), # ytl
                    (row['x'] + row['w']/2), # xbr
                    (row['y'] + row['h']/2)] # ybr
            max_overlap = []
            for crash_frame in crash_frames:
                boxB = [float(crash_frame.attrib.get('xtl')),
                        float(crash_frame.attrib.get('ytl')),
                        float(crash_frame.attrib.get('xbr')),
                        float(crash_frame.attrib.get('ybr'))]
                max_overlap = max(max_overlap, overlap(boxA, boxB))
            if max_overlap > 0.25:
                num_correct += 1
            else:
                num_incorrect += 1

        yolo4_detList.append([num_correct, num_incorrect])
       